# Confusion warning

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import model_selection

**NOTE**: The scorer in `GridSearchCV` can be utterly confusing. 

The scores returned by `GridSearchCV` are negative for scores as `GridSearchCV` by convention tries to maximize its score. This means that loss functions like MSE have to be negated.

See [here](https://stackoverflow.com/questions/21050110/sklearn-gridsearchcv-with-pipeline)
and [here](https://stackoverflow.com/questions/21443865/scikit-learn-cross-validation-negative-values-with-mean-squared-error).

For clarity let's run an experiment.
We will fit a linear regression classifier on data that intersects (0, 1) rather than origo.
Thus we know that including the intercept is better than not.

In [ ]:
def rmse(ground_truth, predictions):
    """
    Returns the root mean squared error of the predictions
    
    The root mean squared error is defined by:
    $\sqrt {\frac {\sum _{t=1}^{T}({\hat {y}}_{t}-y_{t})^{2}}{T}}$
    
    Parameters
    ----------
    ground_truth : array, shape (n_samples,)
        The correct prediction
    prediction : array, shape (n_samples,)
        The predictions
        
    Returns
    -------
    rmse : float
        The root mean squared error
    """
    return np.sqrt(metrics.mean_squared_error(ground_truth, predictions))

In [ ]:
rmse_scorer = metrics.make_scorer(rmse, greater_is_better=False)

In [ ]:
example_train = pd.DataFrame(np.array(range(100)))
example_target = pd.DataFrame(np.array(range(1, 101)))

parameters = {'fit_intercept': (True, False)}

In [ ]:
rmse_scorer_greater = metrics.make_scorer(rmse, greater_is_better=True)
grid_lin_greater = model_selection.GridSearchCV(LinearRegression(), 
                                                parameters,
                                                scoring=rmse_scorer_greater,
                                                return_train_score=False)
grid_lin_greater.fit(example_train, example_target)
greater_best_model = grid_lin_greater.best_estimator_
greater_best_score = grid_lin_greater.best_score_
greater_mean_score = grid_lin_greater.cv_results_['mean_test_score']

In [ ]:
rmse_scorer_lesser = metrics.make_scorer(rmse, greater_is_better=False)
grid_lin_lesser = model_selection.GridSearchCV(LinearRegression(), 
                                               parameters,
                                               scoring=rmse_scorer_lesser,
                                               return_train_score=False)
grid_lin_lesser.fit(example_train, example_target)
lesser_best_model = grid_lin_lesser.best_estimator_
lesser_best_score = grid_lin_lesser.best_score_
lesser_mean_score = grid_lin_lesser.cv_results_['mean_test_score']

In [ ]:
print(f'Greater is best returns best a score of {greater_best_score:.2f} '
      f'of {greater_mean_score} '
      f'with the model\n{greater_best_model}')

In [ ]:
print(f'Lesser is best returns best a score of {lesser_best_score:.2f} '
      f'of {lesser_mean_score} '
      f'with the model\n{lesser_best_model}')